In [1]:
from openai import OpenAI

import json
import minsearch
import openai
import os

In [2]:
client = OpenAI()

In [3]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': "user", "content": "What your name?"}]
)

In [4]:
response.choices[0].message.content

"I'm an AI developed by OpenAI, and I don't have a personal name. You can simply call me Assistant. How can I help you today?"

## Retrivel and Search

In [6]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-28 15:14:35--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642,90K  3,90MB/s    in 0,2s    

2024-06-28 15:14:36 (3,90 MB/s) - ‘documents.json’ saved [658332/658332]



In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
index.fit(documents)

In [9]:
q = 'the course has already started. Can I still enroll?'

In [15]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [16]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generating Answers with OpenAI GPT 4.0

In [13]:
q

'the course has already started. Can I still enroll?'

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': "user", "content": q}]
)

response.choices[0].message.content

"The ability to enroll in a course that has already started can depend on several factors, such as the policies of the institution offering the course, the type of course, and how far along the course is. It's best to check directly with the institution offering the course. Here are a few steps you can take:\n\n1. **Contact the Institution:** Reach out to the admissions office, registrar, or the department offering the course to inquire about late enrollment policies.\n\n2. **Instructor Permission:** Sometimes, enrolling late might require the approval of the course instructor. If so, contact the instructor directly to explain your situation and ask if you can still join the class.\n\n3. **Review Policies:** Check the institution's academic policies regarding deadlines, add/drop periods, and late registration. Often, there are specific windows during which late enrollment is permitted, sometimes with additional fees.\n\n4. **Catch Up on Material:** If you are allowed to enroll late, ma

In [22]:
prompt_template = '''
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only facts from the CONTEXT when answering the qUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}

'''.strip()

In [23]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [26]:
prompt = prompt_template.format(question=q, context=context)
print(prompt)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only facts from the CONTEXT when answering the qUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started. Can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final

In [27]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, you can still enroll in the course even after it has started. You are still eligible to submit the homework, but be aware of the deadlines for submitting the final projects.'